In [ ]:
!pip install jsonlines

In [ ]:
!pip install spacy

In [ ]:
import jsonlines
training_data = []
ent = []

testing_data = []

# Open the JSONL file
with jsonlines.open('/home/abdelrahman/Desktop/Dr. He/Replication-system-demonstration/data/training_data.jsonl') as reader:
    for data in reader:
        text = data['text']
        entities = data['entities']
        for i in entities:
            lb = i['label']
            s = i['start_offset']
            e = i['end_offset']
            ent.append((s,e,lb))
        training_data.append((text, {"entities":ent}))
        ent=[]

# Open the JSONL file
with jsonlines.open('/home/abdelrahman/Desktop/Dr. He/Replication-system-demonstration/data/test.jsonl') as reader:
    for data in reader:
        text = data['text']
        entities = data['entities']
        for i in entities:
            lb = i['label']
            s = i['start_offset']
            e = i['end_offset']
            ent.append((s,e,lb))
        testing_data.append((text, {"entities":ent}))
        ent=[]


In [ ]:
training_data[0]

In [ ]:
testing_data[0]

In [ ]:
import spacy
import random
from spacy.training.example import Example

# Load a blank spacy model
nlp = spacy.blank("en")

# Define the labels (ENTITY types)
labels = ['ITEM', 'EXPERIMNET']

# Add the NER component to the pipeline
ner = nlp.add_pipe('ner', last=True)

# Add labels to the NER component
for label in labels:
    ner.add_label(label)

# initialize the model for training
nlp.begin_training()

# Training loop
for _ in range(200):
    losses = {}
    random.shuffle(training_data)
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.5, losses=losses)
    print("Loss:", losses.get('ner'))

# Save the trained model to disk
nlp.to_disk("/home/abdelrahman/Desktop/Dr. He/Replication-system-demonstration/models/custom_ner_model")


In [ ]:
import spacy
# Load the trained model
nlp = spacy.load("/home/abdelrahman/Desktop/Dr. He/Replication-system-demonstration/models/custom_ner_model")

# Define a function to extract items and experiments from an abstract
def extract_items_and_experiments(abstract_text):
    doc = nlp(abstract_text)
    items = [ent.text for ent in doc.ents if ent.label_ == "ITEM"]
    experiments = [ent.text for ent in doc.ents if ent.label_ == "EXPERIMNET"]
    return items, experiments

# Example abstract text
abstract_text = testing_data[0][0]
# Extract items and experiments
items, experiments = extract_items_and_experiments(abstract_text)

# Print the results
print('text', abstract_text.strip())
print("Items:", items)
print("Experiments:", experiments)